<a href="https://colab.research.google.com/github/aagamaar/Algo_Trading_Python/blob/main/AlgoPy_1ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MOUNTED GOOGLE DRIVE

In [1]:
# Mount Google Drive

from google.colab import drive
drive.mount('/content/drive')
print("Google Drive mounted successfully!")

Mounted at /content/drive
Google Drive mounted successfully!


CREATING PROJECT DIRECTORY STRUCTURE

In [3]:
# Creating Project Directory Structure
import os

# Defining the project root in Google Drive
project_root = '/content/drive/MyDrive/algo_trading_prototype'

# Creating the main project directory
os.makedirs(project_root, exist_ok=True)
print(f"Project root created: {project_root}")

# Creating subdirectories
subdirectories = [
    'config',
    'data',
    'strategy',
    'backtester',
    'analytics',
    'sheets',
    'utils',
    'logs' # Directory for log files
]

for subdir in subdirectories:
    path = os.path.join(project_root, subdir)
    os.makedirs(path, exist_ok=True)
    print(f"Created directory: {path}")

# I have added __init__.py files to make directories into Python packages
# This is crucial for Python to recognize the folders as modules for importing.

for pkg_dir in ['data', 'strategy', 'backtester', 'analytics', 'sheets', 'utils']:
    with open(os.path.join(project_root, pkg_dir, '__init__.py'), 'w') as f:
        pass # Created an empty __init__.py file
    print(f"Created __init__.py in {pkg_dir}")

print("\nDirectory structure created successfully.")

Project root created: /content/drive/MyDrive/algo_trading_prototype
Created directory: /content/drive/MyDrive/algo_trading_prototype/config
Created directory: /content/drive/MyDrive/algo_trading_prototype/data
Created directory: /content/drive/MyDrive/algo_trading_prototype/strategy
Created directory: /content/drive/MyDrive/algo_trading_prototype/backtester
Created directory: /content/drive/MyDrive/algo_trading_prototype/analytics
Created directory: /content/drive/MyDrive/algo_trading_prototype/sheets
Created directory: /content/drive/MyDrive/algo_trading_prototype/utils
Created directory: /content/drive/MyDrive/algo_trading_prototype/logs
Created __init__.py in data
Created __init__.py in strategy
Created __init__.py in backtester
Created __init__.py in analytics
Created __init__.py in sheets
Created __init__.py in utils

Directory structure created successfully.


INSTALLING THE DEPENDENCIES

In [4]:
# Installing the Dependencies
# Used --quiet to suppress verbose output
!pip install pandas numpy ta gspread oauth2client scikit-learn requests yfinance python-telegram-bot --quiet
print("All required Python dependencies installed.")

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 702.3/702.3 kB 22.9 MB/s eta 0:00:00
All required Python dependencies installed.


HOLDS ALL CONFIGURATIONS

In [6]:
# Cell 4: config/settings.py
%%writefile /content/drive/MyDrive/algo_trading_prototype/config/settings.py
# config/settings.py

# Stock Data API (using yfinance)
# No API key is needed for basic yfinance usage
STOCK_SYMBOLS = ["RELIANCE.NS", "TCS.NS", "HDFCBANK.NS"] # Three NIFTY 50 stocks for NSE.

# Google Sheets
GOOGLE_SHEET_ID = "1W8nP1H7oIKwdy7m5dGii0lZcf_cKxn0mPSdzhAddm80"
TRADE_LOG_SHEET_NAME = "Trade Log"
SUMMARY_PL_SHEET_NAME = "Summary P&L"
WIN_RATIO_SHEET_NAME = "Win Ratio"

# Strategy Parameters
RSI_PERIOD = 14
RSI_BUY_THRESHOLD = 30
SHORT_MA_PERIOD = 20 # 20-Day Moving Average
LONG_MA_PERIOD = 50  # 50-Day Moving Average

# Backtesting Parameters
BACKTEST_DURATION_MONTHS = 6

# ML Model Parameters
FEATURES = ['RSI', 'MACD', 'Volume', 'Close'] # Features for ML model
TARGET = 'Next_Day_Movement' # Targets for ML model

# Telegram Alerts (Bonus)
TELEGRAM_BOT_TOKEN = "8144019769:AAF-f7tW-XV9URIgJAAFyQgNtE0Tce0naXw"
TELEGRAM_CHAT_ID = "1463467106"

Overwriting /content/drive/MyDrive/algo_trading_prototype/config/settings.py


SETTING UP LOGGING TO BOTH CONSOLE AND A FILE IN LOGS DIRECTORY

In [9]:
%%writefile /content/drive/MyDrive/algo_trading_prototype/utils/logger.py
# utils/logger.py

import logging
import os
from datetime import datetime

def setup_logging():
    """Configures logging for the application."""
    # Adjusted log_dir to be within your mounted Google Drive project structure.
    log_dir = '/content/drive/MyDrive/algo_trading_prototype/logs'
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    log_file = os.path.join(log_dir, f"algo_trading_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log")

    logging.basicConfig(
        level=logging.INFO, # Log INFO, WARNING, ERROR, CRITICAL messages
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(log_file), # Log to a file
            logging.StreamHandler()        # Also print to console
        ]
    )
    # Suppressing verbose logging from libraries to keep console clean
    logging.getLogger('requests').setLevel(logging.WARNING)
    logging.getLogger('urllib3').setLevel(logging.WARNING)
    logging.getLogger('yfinance').setLevel(logging.WARNING)
    logging.getLogger('gspread').setLevel(logging.WARNING)

Overwriting /content/drive/MyDrive/algo_trading_prototype/utils/logger.py


HANDLES FETCHING STOCK DATA USING YFINANCE

In [11]:
%%writefile /content/drive/MyDrive/algo_trading_prototype/data/data_fetcher.py
# data/data_fetcher.py

import yfinance as yf
import pandas as pd
import logging
from datetime import datetime, timedelta
# Import BACKTEST_DURATION_MONTHS from settings for use in get_historical_data
from config import settings # Import settings module directly

logger = logging.getLogger(__name__)

class DataFetcher:
    def __init__(self):
        # yfinance doesn't require an API key for basic usage
        pass

    def fetch_historical_data(self, symbol: str, start_date: str, end_date: str) -> pd.DataFrame:
        """
        Fetches historical stock data for a given symbol using yfinance.
        Parameters:
            symbol (str): Stock ticker symbol (e.g., "RELIANCE.NS").
            start_date (str): Start date in 'YYYY-MM-DD' format.
            end_date (str): End date in 'YYYY-MM-DD' format.
        Returns:
            pd.DataFrame: DataFrame with historical OHLCV data, or empty if fetching fails.
        """
        try:
            # yf.download returns a DataFrame directly with uppercase columns
            df = yf.download(symbol, start=start_date, end=end_date, progress=False) # progress=False to reduce console output during download
            if df.empty:
                logger.warning(f"No data fetched for {symbol} between {start_date} and {end_date}.")
                return pd.DataFrame()

            # Ensure we have the standard OHLCV columns and index is named 'Date'
            df = df[['Open', 'High', 'Low', 'Close', 'Volume']]
            df.index.name = 'Date'
            logger.info(f"Successfully fetched historical data for {symbol}.")
            return df
        except Exception as e:
            logger.error(f"Error fetching data for {symbol} using yfinance: {e}")
            return pd.DataFrame()

def get_historical_data(symbols: list, duration_months: int) -> dict:
    """
    Fetches historical data for multiple symbols for the last 'duration_months' using yfinance.
    Parameters:
        symbols (list): List of stock ticker symbols.
        duration_months (int): Number of months for which to fetch historical data.
    Returns:
        dict: A dictionary where keys are symbols and values are pandas DataFrames.
    """
    data_fetcher = DataFetcher()
    all_data = {}
    end_date = datetime.now()
    # Calculate start date based on duration_months
    start_date = end_date - timedelta(days=duration_months * 30) # Approximate 30 days per month

    # Format dates as 'YYYY-MM-DD' for yfinance
    start_date_str = start_date.strftime('%Y-%m-%d')
    end_date_str = end_date.strftime('%Y-%m-%d')

    for symbol in symbols:
        df = data_fetcher.fetch_historical_data(symbol, start_date_str, end_date_str)
        if not df.empty:
            all_data[symbol] = df
    return all_data

Overwriting /content/drive/MyDrive/algo_trading_prototype/data/data_fetcher.py


FUNCTIONS TO CALCULATE TECHNICAL INDICATORS LIKE RSI AND MOVING AVERAGES

In [12]:
# Cell 7: strategy/indicators.py
%%writefile /content/drive/MyDrive/algo_trading_prototype/strategy/indicators.py
# strategy/indicators.py

import pandas as pd
import ta # Technical Analysis library

def calculate_rsi(df: pd.DataFrame, window: int = 14) -> pd.Series:
    """
    Calculates Relative Strength Index (RSI).
    Parameters:
        df (pd.DataFrame): DataFrame with 'Close' prices.
        window (int): The window period for RSI calculation.
    Returns:
        pd.Series: A Series containing RSI values.
    """
    return ta.momentum.RSIIndicator(df["Close"], window=window).rsi()

def calculate_sma(df: pd.DataFrame, window: int) -> pd.Series:
    """
    Calculates Simple Moving Average (SMA).
    Parameters:
        df (pd.DataFrame): DataFrame with 'Close' prices.
        window (int): The window period for SMA calculation.
    Returns:
        pd.Series: A Series containing SMA values.
    """
    return ta.trend.SMAIndicator(df["Close"], window=window).sma_indicator()

def calculate_macd(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates Moving Average Convergence Divergence (MACD), Signal Line, and MACD Histogram.
    Parameters:
        df (pd.DataFrame): DataFrame with 'Close' prices.
    Returns:
        pd.DataFrame: DataFrame with 'MACD', 'MACD_Signal', and 'MACD_Hist' columns.
    """
    macd_indicator = ta.trend.MACD(df["Close"])
    return pd.DataFrame({
        'MACD': macd_indicator.macd(),
        'MACD_Signal': macd_indicator.macd_signal(),
        'MACD_Hist': macd_indicator.macd_diff()
    })

Overwriting /content/drive/MyDrive/algo_trading_prototype/strategy/indicators.py


IMPLEMENTS THE TRADING STRATEGY LOGIC (RSI + Moving Average crossover)

In [13]:
# Cell 8: strategy/trading_strategy.py
%%writefile /content/drive/MyDrive/algo_trading_prototype/strategy/trading_strategy.py
# strategy/trading_strategy.py

import pandas as pd
import logging
from strategy.indicators import calculate_rsi, calculate_sma
from config import settings # Import settings module directly

logger = logging.getLogger(__name__)

class TradingStrategy:
    def __init__(self):
        self.rsi_period = settings.RSI_PERIOD
        self.rsi_buy_threshold = settings.RSI_BUY_THRESHOLD
        self.short_ma_period = settings.SHORT_MA_PERIOD
        self.long_ma_period = settings.LONG_MA_PERIOD

    def generate_signals(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Generates buy/sell signals based on RSI and Moving Average Crossover strategy.
        Adds 'RSI', 'SMA_short', 'SMA_long', 'Signal' (1=Buy, -1=Sell, 0=Hold), and 'Position' columns.
        Parameters:
            df (pd.DataFrame): DataFrame with historical OHLCV data.
        Returns:
            pd.DataFrame: DataFrame with added indicator, signal, and position columns.
        """
        if df.empty:
            logger.warning("Empty DataFrame provided to generate_signals.")
            return df

        # Calculate indicators
        df['RSI'] = calculate_rsi(df, self.rsi_period)
        df[f'SMA_{self.short_ma_period}'] = calculate_sma(df, self.short_ma_period)
        df[f'SMA_{self.long_ma_period}'] = calculate_sma(df, self.long_ma_period)

        df['Signal'] = 0 # Default to hold signal
        df['Position'] = 0 # 1 for Long, 0 for Flat (no position)

        # Drop NaN values introduced by indicator calculations before generating signals
        df.dropna(subset=['RSI', f'SMA_{self.short_ma_period}', f'SMA_{self.long_ma_period}'], inplace=True)
        if df.empty:
            logger.warning("DataFrame became empty after dropping NaNs for indicator calculations.")
            return df

        # Buy condition: RSI < 30 AND 20-DMA crosses above 50-DMA
        # Check current day's MA relationship and previous day's relationship for a crossover
        buy_condition = (df['RSI'] < self.rsi_buy_threshold) & \
                        (df[f'SMA_{self.short_ma_period}'] > df[f'SMA_{self.long_ma_period}']) & \
                        (df[f'SMA_{self.short_ma_period}'].shift(1) <= df[f'SMA_{self.long_ma_period}'].shift(1))

        # Sell condition: 20-DMA crosses below 50-DMA (as a simple exit)
        sell_condition = (df[f'SMA_{self.short_ma_period}'] < df[f'SMA_{self.long_ma_period}']) & \
                         (df[f'SMA_{self.short_ma_period}'].shift(1) >= df[f'SMA_{self.long_ma_period}'].shift(1))

        # Apply signals
        df.loc[buy_condition, 'Signal'] = 1
        df.loc[sell_condition, 'Signal'] = -1

        # Determine position based on signals
        # This simulates opening a position on a buy signal and closing on a sell signal.
        # It's a simplified approach for backtesting.
        current_position = 0
        position_list = []
        for i in range(len(df)):
            if df['Signal'].iloc[i] == 1: # Buy signal
                current_position = 1 # Go long
            elif df['Signal'].iloc[i] == -1: # Sell signal
                current_position = 0 # Close position (go flat)
            position_list.append(current_position)

        df['Position'] = position_list
        logger.info("Signals generated successfully.")
        return df

Overwriting /content/drive/MyDrive/algo_trading_prototype/strategy/trading_strategy.py


SIMULATES TRADING STRATEGY BASED ON HISTORICAL DATA

In [15]:

# Cell 9: backtester/backtester.py
%%writefile /content/drive/MyDrive/algo_trading_prototype/backtester/backtester.py
# backtester/backtester.py

import pandas as pd
import logging
from strategy.trading_strategy import TradingStrategy

logger = logging.getLogger(__name__)

class Backtester:
    def __init__(self):
        self.strategy = TradingStrategy()
        self.trade_log = [] # To store details of each trade

    def run_backtest(self, symbol: str, historical_data: pd.DataFrame) -> dict:
        """
        Runs a backtest on the given historical data for a symbol.
        Simulates trades based on signals and calculates P&L.
        Parameters:
            symbol (str): Stock ticker symbol.
            historical_data (pd.DataFrame): DataFrame with historical OHLCV data.
        Returns:
            dict: Dictionary containing backtest summary (initial/final capital, total P&L)
                  and the detailed trade log. Returns empty dict if backtest cannot run.
        """
        if historical_data.empty:
            logger.warning(f"No historical data provided for backtesting {symbol}.")
            return {}

        df = historical_data.copy()
        df = self.strategy.generate_signals(df)

        # Filter out NaN rows that resulted from indicator calculations for cleaner iteration
        df.dropna(subset=['RSI', 'Signal', 'Position'], inplace=True)
        if df.empty:
            logger.warning(f"Data for {symbol} became empty after dropping NaNs for signals. Cannot backtest.")
            return {}

        initial_capital = 100000 # Example initial capital
        current_capital = initial_capital
        position_open = False
        buy_price = 0
        shares_held = 0

        # Reset trade_log for each backtest run
        self.trade_log = []

        for i in range(len(df)):
            date = df.index[i]
            close_price = df['Close'].iloc[i]
            signal = df['Signal'].iloc[i]
            current_position_status = df['Position'].iloc[i] # Current position from strategy

            # Execute buy trade
            if signal == 1 and not position_open: # Buy signal and no open position
                shares_to_buy = int(current_capital / close_price) # Buy as many shares as possible
                if shares_to_buy > 0:
                    buy_price = close_price
                    shares_held = shares_to_buy
                    current_capital -= (shares_held * buy_price) # Deduct cost
                    position_open = True
                    self.trade_log.append({
                        'Symbol': symbol,
                        'Date': date.strftime('%Y-%m-%d'),
                        'Type': 'BUY',
                        'Price': round(buy_price, 2),
                        'Shares': shares_held,
                        'Capital_After_Trade': round(current_capital, 2),
                        'P&L': 0.0 # P&L is realized on sell
                    })
                    logger.info(f"{symbol} - {date.strftime('%Y-%m-%d')}: BUY at {buy_price:.2f} (Shares: {shares_held})")

            # Execute sell trade
            elif signal == -1 and position_open: # Sell signal and there is an open position
                sell_price = close_price
                pnl = (sell_price - buy_price) * shares_held
                current_capital += (shares_held * sell_price) # Add proceeds
                position_open = False
                self.trade_log.append({
                    'Symbol': symbol,
                    'Date': date.strftime('%Y-%m-%d'),
                    'Type': 'SELL',
                    'Price': round(sell_price, 2),
                    'Shares': shares_held,
                    'Capital_After_Trade': round(current_capital, 2),
                    'P&L': round(pnl, 2)
                })
                logger.info(f"{symbol} - {date.strftime('%Y-%m-%d')}: SELL at {sell_price:.2f} (P&L: {pnl:.2f})")
                buy_price = 0 # Reset buy price
                shares_held = 0 # Reset shares held

        # If a position is still open at the end of the backtest, close it forcibly
        if position_open:
            sell_price = df['Close'].iloc[-1]
            pnl = (sell_price - buy_price) * shares_held
            current_capital += (shares_held * sell_price)
            self.trade_log.append({
                'Symbol': symbol,
                'Date': df.index[-1].strftime('%Y-%m-%d'),
                'Type': 'SELL (Forced Exit)', # Indicate a forced exit at end of backtest period
                'Price': round(sell_price, 2),
                'Shares': shares_held,
                'Capital_After_Trade': round(current_capital, 2),
                'P&L': round(pnl, 2)
            })
            logger.info(f"{symbol} - Forced SELL at {sell_price:.2f} (P&L: {pnl:.2f}) at end of backtest.")

        total_pnl = current_capital - initial_capital
        logger.info(f"Backtest for {symbol} completed. Total P&L: {total_pnl:.2f}")

        return {
            'symbol': symbol,
            'initial_capital': initial_capital,
            'final_capital': current_capital,
            'total_pnl': total_pnl,
            'trade_log': pd.DataFrame(self.trade_log) # Convert to DataFrame for easier handling
        }

Overwriting /content/drive/MyDrive/algo_trading_prototype/backtester/backtester.py
